In [13]:
import torch
from torch.utils.data import DataLoader
from pathlib import Path
import torchvision
import pandas as pd

from modeldiff import ModelDiff
from src.data import datasets
from src.data import transforms

## Setup data

In [14]:
_metadata = pd.read_pickle('metadata/living17_val_metadata.df')
class_names = dict(zip(_metadata['class'], _metadata['class_name'].apply(lambda s: s[:10])))

IMAGENET_DIR = '/mnt/cfs/datasets/pytorch_imagenet' 
living17_tf = transforms.LIVING17_TRANSFORMS['val']

dsets = {
    'val': datasets.Living17('val', living17_tf, imagenet_dir=IMAGENET_DIR),
    'train': datasets.Living17('train', living17_tf, imagenet_dir=IMAGENET_DIR)
}

### Helper functions

In [21]:
def vector_to_parameters(vec, parameters) -> None:
    pointer = 0
    for param in parameters:
        num_param = param.numel()
        param.data = vec[pointer:pointer + num_param].view_as(param).data
        pointer += num_param

# for loading checkpoints
def from_npy_to_state_dict(ndarr, model):
    ndarr = torch.from_numpy(ndarr).cuda()
    PARAM_LEN = 11185233
    sd_list = []

    for v in ndarr:
        params = v[:PARAM_LEN].float()
        buffs = v[PARAM_LEN:].float()
        vector_to_parameters(torch.tensor(params), model.parameters())
        vector_to_parameters(torch.tensor(buffs), model.buffers())
        model.eval()
        sd_list.append(model.state_dict().copy())

    return sd_list

### Initialize the models we want to compare.
For simplicity, we will use the same architecture for both models (though this is not necessary).

In [16]:
modelA = torchvision.models.resnet18()
modelA.fc = torch.nn.Linear(512, 17)
modelA.eval()
modelA.cuda() 

modelB = torchvision.models.resnet18()
modelB.fc = torch.nn.Linear(512, 17)
modelB.eval()
modelB.cuda();

### Initialize the training set
Has to be the same for both models.

In [17]:
train_loader = DataLoader(dataset=dsets['train'], batch_size=128, num_workers=3, pin_memory=True)

### Load checkpoints
Load multiple model checkpoints in order to compute TRAK attribution scores. For more details, check out TRAK's [repo](https://github.com/MadryLab/trak) (and TRAK's [quickstart](https://trak.readthedocs.io/en/latest/quickstart.html)).

The expected format is a list of `state_dict`s

In [22]:
# run scripts/download_living17_checkpoints.sh first
all_ckpts = torch.load(Path('../checkpoints/living17.pt'))
ckptsA = from_npy_to_state_dict(all_ckpts['with data aug'], modelA)[:5] # take the ckpts of the first 5 models
ckptsB = from_npy_to_state_dict(all_ckpts['without data aug'], modelB)[:5]

/tmp/ipykernel_713923/3086413736.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vector_to_parameters(torch.tensor(params), model.parameters())
/tmp/ipykernel_713923/3086413736.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vector_to_parameters(torch.tensor(buffs), model.buffers())


### Initialize the `ModelDiff` instance

In [23]:
md = ModelDiff(modelA, modelB, ckptsA, ckptsB, train_loader=train_loader)

                             Report any issues at https://github.com/MadryLab/trak/issues
INFO:STORE:No existing model IDs in /mnt/xfs/home/krisgrg/projects/modeldiff/notebooks/modeldiff_scores/modelA.
INFO:STORE:No existing TRAK scores in /mnt/xfs/home/krisgrg/projects/modeldiff/notebooks/modeldiff_scores/modelA.
                             Report any issues at https://github.com/MadryLab/trak/issues
INFO:STORE:No existing model IDs in /mnt/xfs/home/krisgrg/projects/modeldiff/notebooks/modeldiff_scores/modelB.
INFO:STORE:No existing TRAK scores in /mnt/xfs/home/krisgrg/projects/modeldiff/notebooks/modeldiff_scores/modelB.


### Take any `diff` of choice!

Now you can compute `A-B` and `B-A` with just a single line of code!

In [24]:
val_loader1 = DataLoader(dataset=dsets['val'], batch_size=128, num_workers=3, pin_memory=True)

In [ ]:
diff1 = md.get_A_minus_B(val_loader=val_loader1, num_pca_comps=2)

Once we've initialized the `md` instance, we can use it to compute `diff`s wrt many different target datasets (e.g., like with `val_loader1` above).

In [ ]:
val_loader2 = DataLoader(dataset=dsets['val'][0:127], batch_size=128, num_workers=3, pin_memory=True)
diff2 = md.get_B_minus_A(val_loader=val_loader2, num_pca_comps=4)

### Bring Your Own Scores

If you already have some attribution scores computed, you can still use the same API!

In [13]:
md_from_scores = ModelDiff()

In [16]:
# run scripts/download_living17_checkpoints.sh first
from pathlib import Path

scores_dir = Path('./datamodels/')
scoresA = torch.load(scores_dir.joinpath('living17_data-aug.pt'))['weight']
scoresB = torch.load(scores_dir.joinpath('living17_without-data-aug.pt'))['weight']

In [17]:
diff = md_from_scores.get_A_minus_B_from_scores(scoresA, scoresB, num_pca_comps=2)

In [18]:
diff

{'directions': array([[-0.00066547,  0.00221916,  0.00012629, ...,  0.00020184,
         -0.00036254, -0.00049859],
        [-0.00524965, -0.00065124, -0.00038203, ...,  0.00063359,
         -0.0018108 , -0.00165741]], dtype=float32),
 'projections': array([[ 5.1245297e-04, -2.4516261e-04],
        [-1.0042154e-04,  5.1465724e-04],
        [-3.7063823e-05,  8.0200400e-05],
        ...,
        [-1.7703998e-05,  2.1589889e-04],
        [-8.5801890e-05, -1.4266069e-04],
        [-5.9207014e-05,  5.5971013e-05]], dtype=float32),
 'variances': {'A': array([0.00033447, 0.00109441], dtype=float32),
  'B': array([0.00012626, 0.00037166], dtype=float32)}}

That's it!